# 迭代器

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#简介" data-toc-modified-id="简介-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>简介</a></span></li><li><span><a href="#自定义迭代器" data-toc-modified-id="自定义迭代器-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>自定义迭代器</a></span></li></ul></div>

## 简介

In [1]:
x = [1,2,3]
def f(x):
    return x
f(x)

[1, 2, 3]

In [4]:
for i in x:
    print i

1
2
3


其好处是不需要对下标进行迭代，但是有些情况下，我们**既希望获得下标，也希望获得对应的值**，那么可以将迭代器传给 `enumerate` 函数，这样每次迭代都会返回一组` (index, value)` 组成的元组：

In [12]:
x = [2,4,5]
x_enum = []
for i,n in enumerate(x):
    x_enum.append((i,n))
x_enum

[(0, 2), (1, 4), (2, 5)]

In [13]:
# 迭代器对象必须实现 __iter__ 方法：
x = [2,3,5]
# 迭代器对象
i = x.__iter__()
print i

In [14]:
# __iter__() 返回的对象支持 next 方法
# 返回迭代器中的下一个元素：
i.next()

2

In [15]:
i.next()

3

In [16]:
i.next()

5

In [17]:
# 不存在时就会raise 一个 StopIteration 错误：
i.next()

StopIteration: 

In [18]:
x

[2, 3, 5]

In [19]:
# 这个返回的也是迭代器对象
r = reversed(x)
print r

In [20]:
print r.next()
print r.next()
print r.next()

5
3
2


In [22]:
# 字典对象的 iterkeys, itervalues, iteritems 方法返回的都是迭代器：
x = {'a':1,'b':2,'c':333}
x_iterkeys = x.iterkeys()
x_itervalues = x.itervalues()
x_iteritems = x.iteritems()
print x_iterkeys
print x_itervalues
print x_iteritems

<dictionary-keyiterator object at 0x050CE930>
<dictionary-valueiterator object at 0x050CE840>
<dictionary-itemiterator object at 0x050CE960>


In [24]:
# 迭代器的 __iter__ 方法返回它本身：
print x_iteritems.__iter__()

<dictionary-itemiterator object at 0x050CE960>


In [25]:
x_iteritems.next()

('a', 1)

## 自定义迭代器

In [32]:
# 自定义一个 list 的取反迭代器：
class ReverseListIterator(object):
    def __init__(self,list):
        self.list = list
        self.index = len(list)
    def __iter__(self):
        """返回本身"""
        return self
    def next(self):
        self.index -= 1
        if self.index >= 0:
            return self.list[self.index]
        else:
            raise StopIteration    

In [33]:
x = xrange(10)
for i in ReverseListIterator(x):
    print i,

9 8 7 6 5 4 3 2 1 0


In [34]:
class Collatz(object):
    """
    Collatz 猜想：
        奇数 n：返回 3n + 1
        偶数 n：返回 n / 2
        直到 n 为 1 为止：
    """
    def __init__(self,start):
        self.value = start
    def __iter__(self):
        return self
    def next(self):
        if self.value == 1:
            raise StopIteration
        elif self.value % 2 == 0:
            self.value = self.value/2
        else:
            self.value = 3*self.value + 1
        return self.value            

In [35]:
for x in Collatz(7):
    print x,

22 11 34 17 52 26 13 40 20 10 5 16 8 4 2 1


In [36]:
# 迭代器对象存在这种问题
i = Collatz(7)
for x, y in zip(i, i):
    print x, y

22 11
34 17
52 26
13 40
20 10
5 16
8 4
2 1


In [41]:
# 迭代器和可迭代对象分开处理
# 二叉树的中序遍历实现
class BinaryTree(object):
    def __init__(self,value,left=None,right=None):
        self.value = value
        self.left = left
        self.right = right
    def __iter__(self):
        return InorderIterator(self)
    
class InorderIterator(object):    
    def __init__(self, node):
        self.node = node
        # 使用栈保存结点
        self.stack = []    
    def next(self):
        # 中序遍历
        if len(self.stack) > 0 or self.node is not None:
            while self.node is not None:
                self.stack.append(self.node)
                self.node = self.node.left
            # 中间访问结点，弹栈
            node = self.stack.pop()
            # 指针指向右边
            self.node = node.right
            return node.value
        else:
            raise StopIteration()

In [42]:
tree = BinaryTree(
    left=BinaryTree(
        left=BinaryTree(1),
        value=2,
        right=BinaryTree(
            left=BinaryTree(3),
            value=4,
            right=BinaryTree(5)
        ),
    ),
    value=6,
    right=BinaryTree(
        value=7,
        right=BinaryTree(8)
    )
)

In [43]:
for value in tree:
    print value,

1 2 3 4 5 6 7 8


In [44]:
for x,y in zip(tree, tree):
    print x, y

1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
